In [1]:
import pandas as pd 
import numpy as np 
import numpy.linalg as lalg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.regularizers as rg
from sklearn.model_selection import KFold
from Models import *

#config =  tf.compat.v1.ConfigProto( device_count = {'GPU': 1, 'CPU': 6} )
#sess = tf.compat.v1.Session(config=config)
#tf.compat.v1.keras.backend.set_session(sess)

In [2]:
bike = pd.read_csv("../data/SeoulBikeDataCleaned.csv")
bike.fillna(bike.mean())
x = bike[bike.columns[1:15]].to_numpy()
ox = np.insert(x, 0, 1.0, axis = 1)
y = bike[bike.columns[0]].to_numpy()

In [ ]:
auto_perceptron = Perceptron(ox, y, 0.1, build_fn = Perceptron.build_model)
forward_pcp = auto_perceptron.forward_selection(5000)
backward_pcp = auto_perceptron.backward_elimination(5000)
step_pcp = auto_perceptron.stepwise_regression(5000)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________
[0, 5]
-1.1903612300614075
[0, 5, 14]
-1.190611356957612
[0, 5, 14, 13]
-1.1908615409865018
[0, 5, 14, 13, 12]
-1.1911117821676536
[0, 5, 14, 13, 12, 11]
-1.1913620805206544
[0, 5, 14, 13, 12, 11, 10]
-1.1916124360650984
[0, 5, 14, 13, 12, 11, 10, 9]
-1.1918628488205902
[0, 5, 14, 13, 12, 11, 10, 9, 8]
-1.1921133188067428
[0, 5, 14, 13, 12, 11, 10, 9, 8, 7]
-1.192363846043178
[0, 5, 14, 13, 12, 11, 10, 9, 8, 7, 6]
-1.1926144305495265
[0, 5, 14, 13, 12, 11, 10, 9, 8, 7, 6, 4]
-1.1928650723454286
[0, 5, 14, 13, 12, 11, 10, 9, 8, 7, 6, 4, 3]
-1.1931157714505325
[0, 5, 14, 13, 12, 11, 10, 9, 8, 7, 6, 4, 3, 2]
-1.1933665278844967
[0, 5, 14, 13, 12, 11

In [ ]:
auto_3L = NeuralNet3L(ox, y, build_fn = NeuralNet3L.build_model)
forward_3L = auto_3L.forward_selection()
backward_3L = auto_3L.backward_elimination()
step_3L = auto_3L.stepwise_regression()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                225       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 16        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
[0, 2]
0.29276247852385984
[0, 2, 1]
0.4419472922510489
[0, 2, 1, 3]
0.5293834332483933
[0, 2, 1, 3, 14]
0.6040220936865619
[0, 2, 1, 3, 14, 12]
0.5770801091973086
[0, 2, 1, 3, 14, 12, 11]
0.6545075123499124
[0, 2, 1, 3, 14, 12, 11, 8]
0.6341405548597985
[0, 2, 1, 3, 14, 12, 11, 8, 6]
0.6605951882419471
[0, 2, 1, 3, 14, 12, 11, 8, 6, 7]
0.7620224084786007
[0, 2, 1, 3, 14, 12, 11, 8, 6, 7, 10]
0.7090227333242273
[0, 2, 1, 3, 14, 12, 11, 8, 6, 7, 10, 13]
0.7288233872671677
[0, 2, 1, 3, 14, 12, 11, 8, 6, 7, 10, 13, 4]

In [ ]:
auto_4L = NeuralNet4L(ox, y, build_fn = NeuralNet4L.build_model)
forward_4L = auto_4L.forward_selection()
backward_4L = auto_4L.backward_elimination()
step_4L = auto_4L.stepwise_regression()

In [ ]:
ridge_perceptron = keras.Sequential()
ridge_perceptron.add(layers.Dense(1, input_dim = 15, 
                                 kernel_initializer = "uniform", 
                                 activation = "relu", 
                                 use_bias = False,
                                 kernel_regularizer = rg.l2(0.01)))

optimizer = keras.optimizers.Adam(learning_rate = 0.0005)
ridge_perceptron.compile(loss = "mean_squared_error", optimizer = optimizer)

In [ ]:
ridge_perceptron.fit(ox, y, epochs = 50, batch_size = 10, verbose = 0)
rsq = metrics.rsq(ridge_perceptron, ox, y)
rsq_cv = metrics.rsq_cv(ridge_perceptron, ox, y, epochs = 50)
print(f"Rsq = {rsq} Rsq_cv = {rsq_cv}")

In [ ]:
def plot_and_save(arrays, name, basepath = "../plots/python/"): 
    rsq, rsq_a, rsq_cv, aic = arrays
    x = [_ for _ in range(len(rsq))]
    plt.style.use("fivethirtyeight")
    plt.rcParams["figure.figsize"] = [10,10]
    plt.plot(x, np.array([rsq, rsq_a, rsq_cv]).transpose())
    plt.xlabel("Number of variables")
    plt.ylabel("Rsq Value")
    plt.legend(["Rsq", "RsqAdj", "RsqCV"])
    plt.savefig(basepath+name)
    plt.show()
    
    plt.style.use("fivethirtyeight")
    plt.plot(x, aic)
    plt.xlabel("Number of Variables")
    plt.ylabel("AIC")
    plt.savefig(basepath+"AIC"+name)
    plt.show()
    
plot_and_save(forward_pcp, "AutoForwardPCP.png")
plot_and_save(backward_pcp, "BackWardPCP.png")
plot_and_save(step_pcp, "StepwisePCP.png")

plot_and_save(forward_3L, "AutoForward3L.png")
plot_and_save(backward_3L, "BackWard3L.png")
plot_and_save(step_3L, "Stepwise3L.png")

plot_and_save(forward_4L, "AutoForward4L.png")
plot_and_save(backward_4L, "BackWard4L.png")
plot_and_save(step_4L, "Stepwise4L.png")